In [2]:
import pandas as pd
import matplotlib as rc
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.font_manager as fm
from prophet import Prophet
import numpy as np



In [61]:
file_path="../ref/lotto.xlsx"
ori = pd.read_excel(file_path, engine="openpyxl", header=None)  # 또는 engine="xlrd" (xls인 경우)
ori.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,회차별 추첨결과,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,년도,회차,추첨일,1등,NaN,2등,NaN,3등,NaN,4등,NaN,5등,NaN,당첨번호,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,당첨자수,당첨금액,1,2.0,3.0,4.0,5.0,6.0,보너스
3,2025,1173,2025.05.24,24,"1,179,946,063원",94,"50,210,471원",3139,"1,503,595원",163678,"50,000원",2689832,"5,000원",1,5.0,18.0,20.0,30.0,35.0,3
4,NaN,1172,2025.05.17,13,"2,203,004,452원",98,"48,705,881원",3059,"1,560,372원",152897,"50,000원",2590472,"5,000원",7,9.0,24.0,40.0,42.0,44.0,45


In [62]:

# 필요한 열만 선택해서 직접 이름 지정
df = ori.iloc[3:, 13:20]  # 예: 4번째 줄부터, 3~8번째 열만 사용
df.columns = ['번호1', '번호2', '번호3', '번호4', '번호5', '번호6','보너스']
df = df.dropna().astype(int)  # 또는 applymap(...) 사용 가능
df

,번호1,번호2,번호3,번호4,번호5,번호6,보너스
3,1,5,18,20,30,35,3
4,7,9,24,40,42,44,45
5,3,6,7,11,12,17,19
6,3,13,28,34,38,42,25
7,5,12,24,26,39,42,20
...,...,...,...,...,...,...,...
1171,16,24,29,40,41,42,3
1172,14,27,30,31,40,42,2
1173,11,16,19,21,27,31,30
1174,9,13,21,25,32,42,2


In [63]:
# 모든 번호를 하나의 배열로 평탄화 (flatten)
all_numbers = df.values.flatten()

# 등장 횟수 세기
number_counts = pd.Series(all_numbers).value_counts().sort_values(ascending=False)

# 가장 많이 나온 번호 8개
top8 = number_counts.head(8)

print(top8)


34    203
12    199
27    198
33    197
13    197
17    194
43    194
20    192
Name: count, dtype: int64


In [66]:
number_counts.to_csv("lotto_number_counts.csv", header=["count"])
pd.read_csv("lotto_number_counts.csv").head()

,Unnamed: 0,count
0,34,203
1,12,199
2,27,198
3,33,197
4,13,197


In [72]:
import random
# 기준 CSV 파일 불러오기
file_path = "../ref/lotto_number_counts.csv"  # 파일 경로는 알맞게 수정
df = pd.read_csv(file_path)

# 컬럼명 정리 (번호, 등장횟수)
df.rename(columns={"Unnamed: 0": "number", "count": "count"}, inplace=True)

# 출현 횟수 높은 번호는 확률 낮게, 낮은 번호는 확률 높게 → 역가중치 계산
max_count = df["count"].max()
df["weight"] = max_count - df["count"] + 1  # 최소값 1 이상으로 보정

# 번호 리스트 및 가중치 리스트 준비
numbers = df["number"].tolist()
weights = df["weight"].tolist()

# 추천 숫자 4개 뽑기 (중복 없이)
selected = set()
while len(selected) < 4:
    pick = random.choices(numbers, weights=weights, k=1)[0]
    if pick not in selected:
        selected.add(pick)

# 추가 랜덤 숫자 2개 (중복 허용)
extra = random.sample(range(1, 46), 2)
bonus = extra[-1]

# 결과 출력
print("✅ 추천 번호 4개:", sorted(selected))
print("🎲 추가 번호 2개:", extra)
print("⭐ 보너스 번호:", bonus)

✅ 추천 번호 5개: [4, 5, 8, 29, 42]
🎲 추가 번호 2개: [31, 15]
⭐ 보너스 번호: 15


In [73]:
import pandas as pd
import random

# 파일 경로 설정
file_path = "../ref/lotto_number_counts.csv"  # 경로는 상황에 따라 수정
df = pd.read_csv(file_path)

# 컬럼 이름 정리
df.rename(columns={"Unnamed: 0": "number", "count": "count"}, inplace=True)

# 역가중치 계산 (출현 적은 번호 우선 선택되게)
max_count = df["count"].max()
df["weight"] = max_count - df["count"] + 1

# 번호와 가중치 리스트
numbers = df["number"].tolist()
weights = df["weight"].tolist()

# 결과 저장할 리스트
lotto_sets = []

# 10조합 생성
for _ in range(10):
    selected = set()
    
    # 4개 번호 선택 (중복 없이, 등장 횟수 적은 번호 우선)
    while len(selected) < 4:
        pick = random.choices(numbers, weights=weights, k=1)[0]
        if pick not in selected:
            selected.add(pick)
    
    # 추가 랜덤 번호 2개 (중복 허용)
    extra = random.sample(range(1, 46), 2)
    
    # 6개 번호로 구성 (추천 4개 + 랜덤 2개)
    full_set = sorted(list(selected) + extra)
    lotto_sets.append(full_set)

# 결과 출력
for i, nums in enumerate(lotto_sets, 1):
    print(f"{i}번 조합: {nums}")


1번 조합: [5, 7, 8, 22, 28, 39, 42]
2번 조합: [3, 9, 9, 24, 40, 44, 45]
3번 조합: [2, 22, 22, 32, 39, 44, 45]
4번 조합: [5, 9, 14, 21, 28, 39, 41]
5번 조합: [7, 13, 21, 24, 27, 32, 37]
6번 조합: [5, 5, 10, 11, 22, 24, 35]
7번 조합: [11, 19, 20, 27, 32, 32, 36]
8번 조합: [7, 8, 19, 27, 29, 32, 41]
9번 조합: [9, 24, 28, 37, 39, 39, 41]
10번 조합: [1, 16, 25, 29, 38, 39, 45]


In [74]:
import pandas as pd
import random

# 파일 경로 및 데이터 불러오기
file_path = "../ref/lotto_number_counts.csv"  # 본인 경로에 맞게 수정
df = pd.read_csv(file_path)

# 열 이름 정리
df.rename(columns={"Unnamed: 0": "number", "count": "count"}, inplace=True)

# 등장 횟수 적을수록 확률 높게 설정
max_count = df["count"].max()
df["weight"] = max_count - df["count"] + 1

numbers = df["number"].tolist()
weights = df["weight"].tolist()

# 추천 조합 생성
lotto_sets = []
for _ in range(10):
    selected = set()
    while len(selected) < 4:
        pick = random.choices(numbers, weights=weights, k=1)[0]
        if pick not in selected:
            selected.add(pick)
    extra = random.sample(range(1, 46), 2)
    full_set = sorted(list(selected) + extra)
    lotto_sets.append(full_set)

# DataFrame 변환
lotto_df = pd.DataFrame(lotto_sets, columns=["번호1", "번호2", "번호3", "번호4", "번호5", "번호6"])

# CSV 저장
lotto_df.to_csv("추천로또조합.csv", index=False, encoding="utf-8-sig")

# Excel 저장 (선택)
lotto_df.to_excel("추천로또조합.xlsx", index=False)

# 확인
lotto_df.head()


,번호1,번호2,번호3,번호4,번호5,번호6
0,11,11,15,17,20,38
1,9,9,23,25,32,45
2,9,31,37,41,42,45
3,1,9,11,25,25,44
4,9,10,16,31,32,37
